## **INTRODUCTION**

This project is based on a Hotel dataset gotten from Kaggle, it shows the dataset of the customer that patronise the hotel, the booking methods used, the distribution channels, revenues and loss as well as other measurable indices.

## **QUESTIONS TO BE ANSWERED**

1\. Total revenue made by countries per year

2\. The most profitabale customer category

3\. Relationship between deposit types and cancellation.

## **DIAGRAM OF DIMENSIONS AND FACT**

This diagram shows the dim tables and the fact table as well as their columns, it also shows the relationship between the dim tables and the fact table. 
![DDL DIAGRAM](docs/erd.png)


  

## **CREATION AND USE OF DATABASE**

```
Create database Myproject

use Myproject;
go

```

## **SCHEMA CONSTRUCTION**

Three schemas are going to be created; stg, dim and fa.

```
IF NOT EXISTS (SELECT * FROM sys.schemas WHERE NAME = 'stg')
BEGIN
    EXEC('CREATE SCHEMA stg AUTHORIZATION dbo;');
END;
GO

```
```
IF NOT EXISTS (SELECT * FROM sys.schemas WHERE NAME = 'dim')
BEGIN
    EXEC('CREATE SCHEMA dim AUTHORIZATION dbo;');
END;
GO

```
```
IF NOT EXISTS (SELECT * FROM sys.schemas WHERE NAME = 'fa')
BEGIN
    EXEC('CREATE SCHEMA fa AUTHORIZATION dbo;');
END;
GO

```

## **DIM TABLES CREATION**

After schemas have been created, it is time to create the dim tables (dim.Country, dim.CusType, dim.DepType, dim.Distchan, dim.Hotel\_dimen, Fa.Fact\_Table

```
IF NOT EXISTS (SELECT * FROM INFORMATION_SCHEMA.TABLES WHERE TABLE_SCHEMA = 'dim' AND TABLE_NAME = 'Country')
BEGIN
    CREATE TABLE dim.Country(
    IDCountry bigint NOT NULL,
    Country nvarchar(max) not NULL
    )
    ;

    ALTER TABLE dim.Country
    ADD CONSTRAINT PK_IDCountry PRIMARY KEY(IDCountry);

END

```
```
IF NOT EXISTS (SELECT * FROM INFORMATION_SCHEMA.TABLES WHERE TABLE_SCHEMA = 'dim' AND TABLE_NAME = 'CusType')
BEGIN
    CREATE TABLE dim.CusType(
    ID_CusType bigint NOT NULL,
    Customer_Type nvarchar(max) not NULL
    )
    ;

    ALTER TABLE dim.CusType
    ADD CONSTRAINT PK_ID_CusType PRIMARY KEY(ID_CusType);

END

```
```
IF NOT EXISTS (SELECT * FROM INFORMATION_SCHEMA.TABLES WHERE TABLE_SCHEMA = 'dim' AND TABLE_NAME = 'DepType')
BEGIN
    CREATE TABLE dim.DepType(
    ID_DepType bigint NOT NULL,
    Deposit_Type nvarchar(max) not NULL
    )
    ;

    ALTER TABLE dim.DepType
    ADD CONSTRAINT PK_ID_DepType PRIMARY KEY(ID_DepType);

END

```
```
IF NOT EXISTS (SELECT * FROM INFORMATION_SCHEMA.TABLES WHERE TABLE_SCHEMA = 'dim' AND TABLE_NAME = 'distchan')
BEGIN
    CREATE TABLE dim.distchan(
    ID_Distchan bigint NOT NULL,
    Dist_Channel nvarchar(max) not NULL
    )
    ;

    ALTER TABLE dim.distchan
    ADD CONSTRAINT PK_ID_Distchan PRIMARY KEY(ID_Distchan);

END

```
```
IF NOT EXISTS (SELECT * FROM INFORMATION_SCHEMA.TABLES WHERE TABLE_SCHEMA = 'dim' AND TABLE_NAME = 'Hotel_dimen')
BEGIN
    CREATE TABLE dim.Hotel_dimen(
    ID_Hotel bigint NOT NULL,
    Hotel nvarchar(max) not NULL
    )
    ;

    ALTER TABLE dim.Hotel_dimen
    ADD CONSTRAINT PK_ID_Hotel PRIMARY KEY(ID_Hotel);

END
```

## **FACT TABLE CREATION**

Here's the query to create the fact table, Fa.Fact\_Table and create foreign keys 

```
IF EXISTS (SELECT * FROM INFORMATION_SCHEMA.TABLES WHERE TABLE_SCHEMA = 'fa' AND TABLE_NAME = 'Fact_Table')
BEGIN
    DROP TABLE fa.Fact_Table;
END

GO

    CREATE TABLE fa.Fact_Table(
    Booking_ID bigint Not NULL,
    ID_Hotel bigint NULL,
    Hotel nvarchar(max) NULL,
    Booking_Date datetime2(0) Null,
    Arrival_Date datetime2(0) Null,
    Lead_Time bigint null,
    Nights bigint null,
    Guests bigint null,
    ID_Distchan bigint null,
    Distribution_Channel nvarchar(max) null,
    Customer_Type nvarchar(max) null,
    ID_CusType bigint null,
    Country nvarchar(max) null,
    ID_DepType bigint null,
    Deposit_Type nvarchar(max) null,
    Avg_Daily_Rate float null,
    [Status] nvarchar(max) null,
    Status_Update datetime2(0) null,
    [Cancelled(0/1)] bigint null,
    Revenue float null,
    Revenue_Loss float null,
    IDCountry bigint null
    )
    ;

ALTER TABLE fa.Fact_Table
ADD CONSTRAINT PK_Booking_ID PRIMARY KEY(Booking_ID)
;

ALTER TABLE fa.Fact_Table
ADD CONSTRAINT FK_FatoCou
    FOREIGN KEY (IDCountry)              -- FROM the LOCAL TABLE
     REFERENCES  dim.Country(IDCountry) -- TO the FOREIGN TABLE
;

ALTER TABLE fa.Fact_Table
ADD CONSTRAINT FK_FatoCT
    FOREIGN KEY (ID_CusType)              -- FROM the LOCAL TABLE
     REFERENCES  dim.CusType(ID_CusType) -- TO the FOREIGN TABLE
;

ALTER TABLE fa.Fact_Table
ADD CONSTRAINT FK_FatoDT
    FOREIGN KEY (ID_DepType)              -- FROM the LOCAL TABLE
     REFERENCES  dim.DepType(ID_DepType) -- TO the FOREIGN TABLE
;

ALTER TABLE fa.Fact_Table
ADD CONSTRAINT FK_FatoDC
    FOREIGN KEY (ID_Distchan)
     REFERENCES dim.distchan(ID_Distchan)
;

ALTER TABLE fa.Fact_Table
ADD CONSTRAINT FK_FatoHD
    FOREIGN KEY (ID_Hotel)
     REFERENCES dim.Hotel_dimen(ID_Hotel)
;

```

## **IMPORTING DATA FROM DATABASE, USING SSMS**

Data from the stg tables will be imported into the dim tables.

```
INSERT INTO dim.Country(IDCountry,Country )
	SELECT sc.IDCountry
		  ,sc.Country
	FROM stg.Country sc
	WHERE NOT EXISTS (
    SELECT 1
    FROM dim.Country dc
    WHERE dc.IDCountry = sc.IDCountry
);

GO
```
```
INSERT INTO dim.CusType(ID_CusType,Customer_Type )
	SELECT sct.ID_CusType
		  ,sct.Customer_Type
	FROM stg.CusType sct
	WHERE NOT EXISTS (
    SELECT 1
    FROM dim.CusType dct
    WHERE dct.ID_CusType = sct.ID_CusType
);


GO
```
```
INSERT INTO dim.DepType(ID_DepType,Deposit_Type )
	SELECT sd.ID_DepType
		  ,sd.Deposit_Type
	FROM stg.DepType sd
	WHERE NOT EXISTS (
    SELECT 1
    FROM dim.DepType ddt
    WHERE ddt.ID_DepType = sd.ID_DepType
);

GO

```
```
INSERT INTO dim.distchan(ID_Distchan,Dist_Channel )
	SELECT dc.ID_Distchan
		  ,dc.Dist_channel
	FROM stg.distchan dc
	WHERE NOT EXISTS (
    SELECT 1
    FROM dim.distchan ddc
    WHERE ddc.ID_Distchan = dc.ID_Distchan
);

GO

```
```
INSERT INTO dim.Hotel_dimen(ID_Hotel,Hotel )
	SELECT hd.ID_Hotel
		  ,hd.Hotel
	FROM stg.Hotel_dimen hd
	WHERE NOT EXISTS (
    SELECT 1
    FROM dim.Hotel_dimen dhd
    WHERE dhd.ID_Hotel = hd.ID_Hotel
);

GO
```

## **IMPORTING INTO FACT TABLE**

Data from the stg tables will be imported into the fact table  

```
INSERT INTO fa.Fact_Table(Booking_ID,ID_Hotel,Hotel,Booking_Date,Arrival_Date,Lead_Time,Nights,Guests,
	ID_Distchan,Distribution_Channel,Customer_Type,ID_CusType,Country,ID_DepType,Deposit_Type,Avg_Daily_Rate,
	[Status],Status_Update,[Cancelled(0/1)],Revenue,Revenue_Loss,IDCountry)
	SELECT ft.Booking_ID,
		   ft.ID_Hotel,
		   ft.Hotel,
		   ft.Booking_Date,
		   ft.Arrival_Date,
		   ft.Lead_Time,
		   ft.Nights,
		   ft.Guests,
		   ft.ID_Distchan,
		   ft.Distribution_Channel,
		   ft.Customer_Type,
		   ft.ID_CusType,
		   ft.Country,
		   ft.ID_DepType,
		   ft.Deposit_Type,
		   ft.Avg_Daily_Rate,
		   ft.[Status],
		   ft.Status_Update,
		   ft.[Cancelled(0/1)],
		   ft.Revenue,
		   ft.Revenue_Loss,
		   ft.IDCountry
	FROM stg.Fact_Table ft
	WHERE NOT EXISTS (
    SELECT 1
    FROM fa.Fact_Table f
    WHERE f.Booking_ID = ft.Booking_ID
);

GO

```